In [1]:
import sys
sys.path.append('d://the-recommender/DNN_for_YouTube_Recommendations')

from sample import ml1m_sample
from sample import ml1m_sample_tfrecord
from trainner_cpu import TrainnerCPU
import tensorflow as tf  

##create sample - 读取原始文件构造
sample_x, sample_y = ml1m_sample.Ml1mSample().ranking_sample()
train_dataset = ml1m_sample_tfrecord.Ml1mTFRecordSample().tfrecord_sample(sample_x[:10000], sample_y[:10000], True) #.tfrecord_sample() #从tfrecord读取样本

##直接从tfrecord文件读取
#train_dataset = ml1m_sample_tfrecord.Ml1mTFRecordSample().tfrecord_sample() 

count = train_dataset.take(-1).reduce(tf.constant(0), lambda x, _: x + 1).numpy()  
print(count)


10000


In [2]:
next(iter(train_dataset))  

({'user_id': <tf.Tensor: shape=(), dtype=int64, numpy=1>,
  'item_id': <tf.Tensor: shape=(), dtype=int64, numpy=48>,
  'timestamp': <tf.Tensor: shape=(), dtype=int64, numpy=978824351>,
  'title': <tf.Tensor: shape=(), dtype=string, numpy=b'Pocahontas (1995)'>,
  'genres': <tf.Tensor: shape=(), dtype=string, numpy=b"Animation|Children's|Musical|Romance">,
  'gender': <tf.Tensor: shape=(), dtype=string, numpy=b'F'>,
  'age': <tf.Tensor: shape=(), dtype=int64, numpy=1>,
  'occupation': <tf.Tensor: shape=(), dtype=int64, numpy=10>,
  'zip': <tf.Tensor: shape=(), dtype=string, numpy=b'48067'>,
  'itemid_lastN': <tf.Tensor: shape=(), dtype=string, numpy=b'48|1907|783|1|527'>,
  'itemid_lastN_len': <tf.Tensor: shape=(), dtype=int64, numpy=5>},
 {'rating': <tf.Tensor: shape=(), dtype=int64, numpy=5>})

In [3]:

## 以下输入信息需要和模型的图对比，看看输入输出shape是否匹配
input_tensor_dict = TrainnerCPU().build_input_tensor(train_dataset)
input_tensor_dict

{'onehot_rating_label_tensor': <tf.Tensor: shape=(10000, 6), dtype=float32, numpy=
 array([[0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        ...,
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.]], dtype=float32)>,
 'rating_score_tensor': <tf.Tensor: shape=(10000,), dtype=float32, numpy=array([5., 4., 4., ..., 3., 3., 4.], dtype=float32)>,
 'hashed_userid_tensor': <tf.Tensor: shape=(10000,), dtype=int64, numpy=array([2911, 2911, 2911, ..., 6748, 6748, 6748], dtype=int64)>,
 'hashed_itemid_tensor': <tf.Tensor: shape=(10000,), dtype=int64, numpy=array([1337, 3691, 8599, ..., 2509, 6884, 7231], dtype=int64)>,
 'hashed_zip_tensor': <tf.Tensor: shape=(10000,), dtype=int64, numpy=array([1298, 1298, 1298, ..., 5108, 5108, 5108], dtype=int64)>,
 'tags_tensor': <tf.Tensor: shape=(10000, 310), dtype=float32, numpy=
 array([[0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
  

In [4]:
#训练模型
model = TrainnerCPU().train_loop(train_dataset)


125/125 [==============================] - 10s 68ms/step - loss: 2.5328 - score_loss: 1.0652 - rating_loss: 1.4676 - val_loss: 2.5391 - val_score_loss: 1.0728 - val_rating_loss: 1.4662


In [5]:
## 默认签名
#保存模型
TrainnerCPU().save_model(model)

INFO:tensorflow:Assets written to: d://the-recommender/DNN_for_YouTube_Recommendations/core/trainner/__SavedModel/signature_with_multiple_input_output/assets


INFO:tensorflow:Assets written to: d://the-recommender/DNN_for_YouTube_Recommendations/core/trainner/__SavedModel/signature_with_multiple_input_output/assets


In [6]:
#查看模型签名
model_path="d://the-recommender/DNN_for_YouTube_Recommendations/core/trainner/__SavedModel/signature_with_multiple_input_output"
imported_with_signatures = tf.saved_model.load(model_path)
list(imported_with_signatures.signatures.keys())

['serving_default']

In [7]:
imported_with_signatures.signatures.keys()

KeysView(_SignatureMap({'serving_default': <ConcreteFunction (*, genres: TensorSpec(shape=(None, None), dtype=tf.float32, name='genres'), movie_id: TensorSpec(shape=(None, None), dtype=tf.float32, name='movie_id'), rated_movies_lastN: TensorSpec(shape=(None, None), dtype=tf.float32, name='rated_movies_lastn'), tags: TensorSpec(shape=(None, None, 310), dtype=tf.float32, name='tags'), user_id: TensorSpec(shape=(None, None), dtype=tf.float32, name='user_id'), zip_id: TensorSpec(shape=(None, None), dtype=tf.float32, name='zip_id')) -> Dict[['rating', TensorSpec(shape=(None, None, 6), dtype=tf.float32, name='rating')], ['score', TensorSpec(shape=(None, None, 1), dtype=tf.float32, name='score')]] at 0x179C9965350>}))

In [8]:
## 自定义签名
#保存模型
TrainnerCPU().save_model_signatures(model)

INFO:tensorflow:Assets written to: d://the-recommender/DNN_for_YouTube_Recommendations/core/trainner/__SavedModel/multi_signature_with_multiple_input_output/assets


INFO:tensorflow:Assets written to: d://the-recommender/DNN_for_YouTube_Recommendations/core/trainner/__SavedModel/multi_signature_with_multiple_input_output/assets


In [9]:
#查看模型签名 - 多签名（结果预测和取特征）
model_path="d://the-recommender/DNN_for_YouTube_Recommendations/core/trainner/__SavedModel/multi_signature_with_multiple_input_output"
imported_with_signatures = tf.saved_model.load(model_path)
list(imported_with_signatures.signatures.keys())

['serving_default', 'predict', 'extract_features']

In [10]:
imported_with_signatures.signatures.keys()

KeysView(_SignatureMap({'serving_default': <ConcreteFunction (*, genres: TensorSpec(shape=(None,), dtype=tf.float32, name='genres'), movie_id: TensorSpec(shape=(None,), dtype=tf.float32, name='movie_id'), rated_movies_lastN: TensorSpec(shape=(None,), dtype=tf.float32, name='rated_movies_lastn'), tags: TensorSpec(shape=(None, 310), dtype=tf.float32, name='tags'), user_id: TensorSpec(shape=(None,), dtype=tf.float32, name='user_id'), zip_id: TensorSpec(shape=(None,), dtype=tf.float32, name='zip_id')) -> Dict[['output_1', TensorSpec(shape=(None, 6), dtype=tf.float32, name='output_1')], ['output_0', TensorSpec(shape=(None, 1), dtype=tf.float32, name='output_0')]] at 0x179DA882D50>, 'predict': <ConcreteFunction (*, genres: TensorSpec(shape=(None,), dtype=tf.float32, name='genres'), movie_id: TensorSpec(shape=(None,), dtype=tf.float32, name='movie_id'), rated_movies_lastN: TensorSpec(shape=(None,), dtype=tf.float32, name='rated_movies_lastn'), tags: TensorSpec(shape=(None, 310), dtype=tf.floa

In [11]:
## 推理验证签名
# 使用签名进行推理  
# 获取签名  #hashed_userid_tensor,hashed_itemid_tensor,genres_tensor,itemid_seq_tensor,tags_tensor,rating_score_tensor,onehot_rating_label_tensor
serve_signature = imported_with_signatures.signatures['serving_default']  
serve_signature


<ConcreteFunction (*, genres: TensorSpec(shape=(None,), dtype=tf.float32, name='genres'), movie_id: TensorSpec(shape=(None,), dtype=tf.float32, name='movie_id'), rated_movies_lastN: TensorSpec(shape=(None,), dtype=tf.float32, name='rated_movies_lastn'), tags: TensorSpec(shape=(None, 310), dtype=tf.float32, name='tags'), user_id: TensorSpec(shape=(None,), dtype=tf.float32, name='user_id'), zip_id: TensorSpec(shape=(None,), dtype=tf.float32, name='zip_id')) -> Dict[['output_1', TensorSpec(shape=(None, 6), dtype=tf.float32, name='output_1')], ['output_0', TensorSpec(shape=(None, 1), dtype=tf.float32, name='output_0')]] at 0x179DA882D50>